### Scrape wikipedia for Toronto city data

In [1]:
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

In [3]:
soup = BeautifulSoup(page,'lxml')

In [41]:
#print(soup.prettify())

In [42]:
all_tables=soup.find_all("table")

In [43]:
my_table=soup.find('table', class_='wikitable sortable')

Extract data from table into empty lists

In [33]:
A=[]
B=[]
C=[]


for row in my_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

Create dataframe with respective columns

In [7]:
import pandas as pd
df=pd.DataFrame(A, columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C


In [8]:
# remove '\n'

df['Neighbourhood'] = df['Neighbourhood'].str.replace("\n", "")
df['Borough'] = df['Borough'].str.replace("\n", "")
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Ignore cells with a borough that is Not assigned.

In [9]:
df = df[df.Borough !='Not assigned']

More than one neighborhood can exist in one postal code area. <br>
These two rows will be combined into one row with the neighborhoods separated with a comma

In [10]:
df1 = df.groupby("Postcode").agg(lambda x:', '.join(set(x)))
df1.reset_index(inplace=True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 


In [28]:
df1[df1.Neighbourhood =='Not assigned'] = df1[df1.Neighbourhood =='Not assigned'].Borough

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [32]:
df1.shape

(103, 3)